# Test ETradeBot

In [1]:
import ast
import sys
import keyring

In [2]:
sys.path.insert(0, '../')
from strategies.strategy import Strategy
from etrade.etrade import ETrade
from execute.execute import Execute
from bot import Bot

## Initialize

Log in to E-Trade, authenticate account, and initialize API objects.

In [3]:
# Retrieve credentials from keyring
consumer_key = keyring.get_password("etrade", "consumer_key")
consumer_secret = keyring.get_password("etrade", "consumer_secret")
sandbox_key = keyring.get_password("etrade", "sandbox_key")
sandbox_secret = keyring.get_password("etrade", "sandbox_secret")
web_username = keyring.get_password("etrade", "web_username")
web_password = keyring.get_password("etrade", "web_password")
account_id = keyring.get_password("etrade", "account_id")
etrade_cookie = ast.literal_eval(keyring.get_password("etrade", "cookie"))
account_id_key = keyring.get_password("etrade", "account_id_key")

In [4]:
# Define the strategy name
strategy_name = 'example_strategy'

In [5]:
# Configuration
dev=False  # Set to False for production, True for sandbox
headless=True  # Set to False to show browser window, True to hide
browser='chrome'  # Set to 'chrome' or 'edge'
preview=True  # Set to False to execute trades, True to preview
prints=True  # Set to True to print information to console

In [ ]:
# Initialize API objects
etrade = ETrade(
    consumer_key, consumer_secret, web_username, web_password, account_id, etrade_cookie,
    sandbox_key, sandbox_secret, dev, headless, browser
)
strategy = Strategy(prints, strategy_name)
execute = Execute(etrade)

## Strategy

Run the custom strategy.

In [ ]:
new_portfolio = strategy.run_strategy()

In [ ]:
new_portfolio

## Current Portfolio

Get the current portfolio and buying power of the account from E-Tade.

In [ ]:
# Get current portfolio
current_portfolio = etrade.get_portfolio_data(account_id_key)[[
    'pctOfPortfolio', 'quantity', 'positionType'
]]
current_portfolio

In [ ]:
# Get buying power
buying_power = etrade.get_buying_power(account_id_key, prints=prints)

## Implement Strategy

Generate trades which will implement the custom strategy given the current portfolio. 

In [ ]:
# Calculate new portfolio shares
new_portfolio_shares = execute.calculate_shares(new_portfolio, buying_power, prints=prints)

In [ ]:
# Preview or execute trades
print('')
if preview:
    print('Previewing trades...')
else:
    print('Executing trades...')
place_order = execute.execute_trades(
    current_portfolio, new_portfolio_shares, account_id_key, preview, prints
)

In [ ]:
place_order